In [1]:
import numpy as np
import os
import pandas as pd
import requests
import json
#设置文件路径
output_path = r'F:\yunic\result'
file_path = r'F:\yunic\data'
os.chdir(file_path)
data=pd.read_excel("test_data.xlsx",header=0)
os.chdir(output_path)

In [2]:
data.head(1)

,小说类别,文本来源,输入文本（100字-300字）,小说风格（通过GPT提取）,输出字数（自己输，200-500字，取100整数倍）,续写指令（通过GPT提取）,GPT结果
0,当代文学,《第七天》,我眼睁睁看着病魔一点点地吞噬我父亲的生命，我却无能为力。放疗、手术、化疗，把我曾经强壮的父亲...,"""词汇选择 - 使用具体而生动的描述，强化了疾病对生命侵蚀和个体无力感的情感渲染。\n句法结...",400,在父亲的病床前，我痛不欲生，一个意外的电话却打断了我的沉思。原来是多年未见的舅舅，他声称有一...,我拿起电话，耳边传来舅舅那略带沙哑的声音：“我知道你们现在的情况很困难，我这有个朋友是老中医...


In [3]:
column_names = ["剧情模型结果1", "剧情模型结果2", "剧情模型结果3"]  # 可以根据需要添加更多列名  
add_len=len(column_names)
new_columns = pd.DataFrame(index=data.index, columns=column_names) 
data = pd.concat([data, new_columns], axis=1) 
data.head(1)

,小说类别,文本来源,输入文本（100字-300字）,小说风格（通过GPT提取）,输出字数（自己输，200-500字，取100整数倍）,续写指令（通过GPT提取）,GPT结果,剧情模型结果1,剧情模型结果2,剧情模型结果3
0,当代文学,《第七天》,我眼睁睁看着病魔一点点地吞噬我父亲的生命，我却无能为力。放疗、手术、化疗，把我曾经强壮的父亲...,"""词汇选择 - 使用具体而生动的描述，强化了疾病对生命侵蚀和个体无力感的情感渲染。\n句法结...",400,在父亲的病床前，我痛不欲生，一个意外的电话却打断了我的沉思。原来是多年未见的舅舅，他声称有一...,我拿起电话，耳边传来舅舅那略带沙哑的声音：“我知道你们现在的情况很困难，我这有个朋友是老中医...,NaN,NaN,NaN


In [9]:
data=pd.read_excel(r"F:\yunic\data\test_data.xlsx",header=None)
for index,row in data.iterrows():
    if index==0:
        print(str(row[2]))
        print(str(row[3]))
        print(str(row[4]))
        print(str(row[5]))

输入文本（100字-300字）
小说风格（通过GPT提取）
输出字数（自己输，200-500字，取100整数倍）
续写指令（通过GPT提取）


In [4]:
for index, row in data.iterrows():
    for i in range(add_len):
#         print(str(row[2]))
#         print(str(row["输入文本（100字-300字）"]))
        input_para = str(row[2])
        content_style = str(row[3])
        para_word_counts=int(row[4])
        content_instruction=str(row[5]) 
        prompt = """我需要你帮我写一部小说。每一次，我都会给你当前的记忆（以前的故事的简要总结。你应该用它来存储所写内容的关键内容，这样你就能记下很长的上下文），之前写的段落，小说风格设定以及下一段要写的内容的指示 。你需要根据[小说风格]的内容，逐步思考，只模仿和使用提到的创作方法。严格遵循[小说风格]中的内容进行创作。\n\n我需要你来写：\n输出段落：遵循输入指令生成的小说的下一个段落。应包含大约{para_word_counts}个中文字符。\n\n下面是输入的内容： \n\n输入段落：\n{input_paragraph}\n\n输入指令：\n{input_instruction}。\n\n[小说风格]: {novel_style}\n\n非常重要！! ！请记住你正在写一本小说, 像小说家一样创作,这只是完整小说的其中一个部分，剧情进展不要太快，但需要在适当的节点引入一些悬念或者爽点, 增加对读者对吸引力。 在写小说下一段时, 请考虑哪些情节对读者有吸引力, 并且保证它和上一段的内容连贯。原则: \n不要输出任何续写内容外的东西，如 `输出段落`、`段落字数`等，否则会有严厉的惩罚。\n\n\n\n段落内容:\n"
    """.format(para_word_counts=para_word_counts, input_paragraph=input_para, input_instruction=content_instruction, novel_style=content_style)

        # ---调用gpt api得到结果---
#         print(prompt)
        url = "http://39.107.82.174:21003/v1/chat/completions" 
        payload = json.dumps({
            "model": "string", 
            "messages": [ 
                {
                 "role": "user",
                 "content": prompt
                }
            ], 
            "do_sample": True,
            "temperature": 1,
            "top_p": 0.85,
            'top_k': 10,
            "max_tokens": 2048,
            'repetition_penalty': 1.15,
            "stream": False
        }) 
        headers = { 'User-Agent': 'Apifox/1.0.0 (https://apifox.com)', 
                    'Content-Type': 'application/json'
        } 
        response = requests.request("POST", url, headers=headers, data=payload) 
#         print(response)
#         获取json数据
        json_data = response.json()
        # 提取content字段  
        content = json_data['choices'][0]['message']['content']  
        length=str(len(content))
        print(f"第{index+1}个输入文本的第{i+1}次输出,输出内容是:{content}"+"("+length+")\n")
        
        data.at[index,column_names[i]]=content+"("+length+")"
#         if i==0:
#             data.at[index,"剧情模型结果1"]=content+"("+length+")"
#         elif i==1: 
#             data.at[index,"剧情模型结果2"]=content+"("+length+")"
#         else:
#             data.at[index,"剧情模型结果3"]=content+"("+length+")"


第1个输入文本的第1次输出,输出内容是:虽然心中充满着疑虑，我还是决定试一试。
    那天是星期一的下午，我陪父亲来到位于市郊的这座医院。这是一栋四层楼高的老式建筑，在周围低矮房屋的映衬下，显得有些鹤立鸡群。然而，这栋老建筑却以一种古朴的方式，昭示着它独特的韵味。
    四楼的走廊很窄，只够两人并肩而行。走廊的尽头是两间诊室，一间是老中医的，另一间则是主治医生的。老中医的诊室非常狭小，只放得下一张诊桌和一把椅子。诊桌的左面是药柜，药柜的左边堆放着许多资料和书籍。(215)

第1个输入文本的第2次输出,输出内容是:父亲躺在病床上，半死不活的，我拿着电话的手在发抖，犹豫了，如果父亲因此而耽误了最佳的治疗时间，我……我真的无法想象。我该怎么办？(64)

第1个输入文本的第3次输出,输出内容是:
    那天是星期二，我清楚地记得，星期二是父亲生命最后延续的日子。
    那天，我们按照地址，从南京来到了苏州。(59)

第2个输入文本的第1次输出,输出内容是:过了一会，我正忙着侍弄那些该死的菜，听见了叫声，我的手一抖，差点没捏住那瓮。”她往那小山嘴上一指。
  “那声音是从那边传来的。”梁奶奶说。
  “阿毛！”村里的人立刻意识到发生了什么事。
  “快去找呀！”一个村民说。
  “你一个人去不够，阿毛一定害怕，他会躲起来的。”另一个村民说。
  “可是我们……我们怕……”
  “不要怕！大家一起去，人多胆子大！”
  于是，一群人往山上冲去。
  梁奶奶接着说：“我也想跟去，但是那瓮太重，我搬不动。我只好一个人先去。大家冲进树林的时候，我看见那野狼把阿毛的半个身子已经吞下去了。啊呀，我真后悔叫阿毛一个人去，我真后悔啊……”
  “奶奶！”
  梁奶奶擦了擦眼泪，说：“阿毛的尸体，是找到的，可惜，肠子都已经被吃了。我们把它从狼肚子里掏出来，放到我们家的锅里煮了两个晚上，才洗干净，用盐水腌了。”
  “奶奶，阿毛是谁啊？”
  “就是你的邻居啊。你一睁开眼睛，就什么都不知道了。唉，孩子，”梁奶奶摸摸孙子的头，“你只要记住，春天，也有野兽来吃人的。(451)

第2个输入文本的第2次输出,输出内容是:我叫他中午去吃饭，他却说还没好，我说再不回去吃饭就要饿坏了，他才回来。他说豆子是要给老财家的媳妇吃，说是阿毛他阿大的嫂子生了个儿子。我听了高兴得不得了，连声夸他。”
  

第12个输入文本的第1次输出,输出内容是:篱笆上面原本有一个很大的横匾，写着“风紧夜深”四个字，是舅祖父的墨迹。这四个字，本来是写给住在前院西厢房里的舅祖母看的，因为舅舅是夜里起床上茅房，所以要等白天把门打开，篱笆上面的横匾，其实是给舅祖母看的。可是，这几个字表妹的舅舅却看不清楚，因为门太小了，所以舅祖父就叫表妹的父亲来重新写一个，表妹的父亲不干，舅祖父就叫表妹的父亲“要么重新写，要么叫你老婆看字”，因此，舅祖母只好叫表妹的父亲重新写一个。表妹的父亲就写了“风紧夜深”四个字，但是字迹不像舅祖父写得那样潇洒，不过表妹的舅舅倒是看了。过了没有多久，舅祖父又叫表妹的父亲把门打开，重新写了几个字，让舅祖母看，写了“风平夜静”四个字，意思是说：“你看，现在夜里不正是很安静吗？” 
  可是，舅祖母说：“不，不是风平夜静，是风紧夜深，你怎么能说成是风平夜静呢？(358)

第12个输入文本的第2次输出,输出内容是:那竹篱外面，本来有一道白色泥墙，篱笆上面的竹梢，也都从墙上拔去，远远看去，像是一面倒挂的镜子，映着天上的浮云。现在泥墙不见了，篱笆上面的竹梢，也全被舅母拔去。表妹说，陈家的小孩把篱笆上面的竹子拔去，是因为他们说，舅母家的篱笆太高，挡了他们家的光。
    表妹说着，忽然哭起来。(139)

第12个输入文本的第3次输出,输出内容是:表妹说陈家的孩子们真是没道理，明明是自家的扁豆藤爬到别人家的竹篱上去，陈家竟然说是舅母家的人故意把竹篱弄坏了，还挖了她的扁豆根，真是气人。
  舅母听表妹说了，立刻说要把篱笆修整一下，不让扁豆藤再爬过去，否则就要去陈家讲道理。(113)

第13个输入文本的第1次输出,输出内容是:过了没多久，林哲终于感觉到一股若有若无的气息靠近，他立刻停下所有动作，静心等待，过了没多久，一股若有若无的灰色气流悄无声息地飘到林哲身前，似乎是什么物体散发出的气息。
  “这股气息……难道是？”林哲脑海中灵光闪过，突然察觉到这股气息竟然和刚才那群人身上散发出的气息相似，只是要略微淡一些。
  不过虽然察觉到这一点，林哲却不敢轻举妄动，他察觉到这股气息似乎有种若有若无的牵引力，似乎在引导他往某个地方。
  林哲决定顺着这股气息走，利用修为感应微妙的变化，很快，他感觉到一股若有若无的牵引力从这股气息中发出，似乎有什么东西在吸引着他。
  林哲再次跟随这股牵引力，悄无

In [5]:
data

,小说类别,文本来源,输入文本（100字-300字）,小说风格（通过GPT提取）,输出字数（自己输，200-500字，取100整数倍）,续写指令（通过GPT提取）,GPT结果,剧情模型结果1,剧情模型结果2,剧情模型结果3
0,当代文学,《第七天》,我眼睁睁看着病魔一点点地吞噬我父亲的生命，我却无能为力。放疗、手术、化疗，把我曾经强壮的父亲...,"""词汇选择 - 使用具体而生动的描述，强化了疾病对生命侵蚀和个体无力感的情感渲染。\n句法结...",400,在父亲的病床前，我痛不欲生，一个意外的电话却打断了我的沉思。原来是多年未见的舅舅，他声称有一...,我拿起电话，耳边传来舅舅那略带沙哑的声音：“我知道你们现在的情况很困难，我这有个朋友是老中医...,虽然心中充满着疑虑，我还是决定试一试。\n 那天是星期一的下午，我陪父亲来到位于市郊的...,父亲躺在病床上，半死不活的，我拿着电话的手在发抖，犹豫了，如果父亲因此而耽误了最佳的治疗时间...,\n 那天是星期二，我清楚地记得，星期二是父亲生命最后延续的日子。\n 那天，我...
1,当代文学,《祝福》,“我真傻，真的，”她说。“我单知道雪天是野兽在深山里没有食吃，会到村里来;我不知道春天也会有...,"""词汇选择 - 运用具体生活场景和细腻情感的词汇，营造了一种朴实无华、充满温情的氛围。\n句...",200,“春光明媚的日子并不总是安宁的，村子边缘那条看似平静的小路也潜伏着危机。”接下去的剧情，一只...,那天，春风拂面，阿毛一心一意地剥着豆。村口的黄狗突然“汪汪”叫起来，但村里人都习以为常，没人...,过了一会，我正忙着侍弄那些该死的菜，听见了叫声，我的手一抖，差点没捏住那瓮。”她往那小山嘴上...,我叫他中午去吃饭，他却说还没好，我说再不回去吃饭就要饿坏了，他才回来。他说豆子是要给老财家的...,那条小路很窄，只够一个人通过。如果有什么东西从路那头过来，是很容易看到的。可是这天早晨大家都...
2,都市言情,《千金归来：追妻365天》,心里咯噔一下，糟了，他不是加班吗，怎么这么快出来了。赵竟安看见韩宇由远及近，心跳也越来越激烈...,"""词汇选择 - 通过使用“咯噔一下”和“心跳也越来越激烈”等生动的词汇，营造了一种紧张且急切...",200,赵竟安在焦急中发现，她随意躲藏的那辆车，竟然是她某位不太熟悉但又位高权重的同事的车，而这位同...,赵竟安突然感到一股紧张的气氛，心里直跳。就在这时，那位同事不经意间走到车边，看到了她。那人是...,而那位同事正好在那车里。赵竟安低着头，站在车边，装作在等车的样子。她听见韩宇在她旁边停下了车...,等她回头再想找那位同事的车时，却发现那位同事的车已经到了她前面，并且停下了，同事正一脸惊讶的...,赵竟安突然有种想要哭的冲动，她挪到车边，还没来得及哭出来，车就开了，她的屁股险险的蹭到了车尾...
3,都市言情,《重生娱乐圈：隐婚老公太心计》,身为欧氏集团主管影视的大小姐，她见多了倾国绝色的美女，但没有一个女星能像莫俏俏一样，面容只是...,"""词汇选择 - 通过精心挑选温和且描绘细腻的词汇，创造出既柔和又引人入胜的人物形象。\n句法...",300,大小姐在偶然的场合发现，莫俏俏实际上是自己多年未见的青梅竹马，两人的再相遇掀起了复杂的心绪和...,在一次行业内的盛宴上，大小姐无意间瞥见了一个熟悉的身影。她走近一看，竟是莫俏俏。时间仿佛在那...,\n 两人从青梅竹马到长大后的多年未见，兜兜转转，命运的安排让他们再次成为欢喜冤家。\...,\n 然而，正当大小姐对往事追悔莫及，她万万没想到，在追悼过去的同时，竟然一脚踩进沼泽...,而那个在多年前的幽暗小巷中，救过她一命的莫俏俏，如今竟成了她公司力捧的新星，两人之间的距离拉...
4,古典文学,《唐史并不如烟6——元和中兴》,公元785年正月一日，李适大赦天下，这既是李适给天下不良分子发的新年红包，同 时也包含着李适...,"""词汇选择 - 运用特定的历史地名和官职称谓，展现了故事背景的历史深度和人物地位的变化。\n...",200,在李适擢升卢杞为吉州长史之后，卢杞接到调令的消息心中难以平静。虽然他的权势地位将得到明显提升...,"卢杞收到调令的那一刻，他的脸上不由自主地泛起了复杂的表情。""这份提升，是福是祸？""他自语。刚...",但卢杞明白，他这次升迁是由安国功构成，他必须夹着尾巴做人。\n 第二步，将卢杞调入朝廷担任...,接下来，李适又擢升卢杞为中书令，位列三公，至此，卢杞终于出 了头。 然而，卢杞的春风得意也是...,但同时他也知道，在前往吉州的路 上，他所要面对的决不是鲜花与掌声，而是无数双盯着他的眼睛，其...
5,古典文学,《魏晋南北朝史》,"府兵的前身,是以贺拔岳武川军团和侯莫陈悦军团的一部分(即李弼军团)以及“六坊之众”随魏孝武帝...","""词汇选择 - 通过使用历史人物及地名的专有名词，营造出浓厚的历史文化氛围。\n句法结构 -...",200,经过深思熟虑，根据剧情文本中提到的军队背景，我们将创作的故事发展设定在战乱纷争的古代背景下。...,在一个寂静的夜晚，三军营帐之内，火光闪烁，映出几位将军的脸庞，他们细声商谈着什么。一个中年将...,"但到头来,随着魏孝武帝死后六国形势的变化,这三个军团的未来却并不相同。其中,贺拔岳武川军团由...","但后来由于南北朝军阀混战,时无宁日,府兵便趁机迅速发展起来,最终取代了北魏,成了割据一方的军...","后来,随着孝武帝的南迁,这些军队逐渐游离散居在北方各地,其中最南端的两个集团(即贺拔岳军团和..."
6,科幻小说,《末日乐园》,“可是那也不能解释她为什么不喝水啊？”卢泽回想起刚才几个人挨挨碰碰地在楼梯间里一块儿下楼的情...,"""词汇选择 - 通过使用具体的描述和假设性的语言，营造了一种紧张且充满好奇的氛围。\n句法结...",200,卢泽思索着玛瑟的猜测，决定暗中观察那位神秘女子的一举一动，并寻找机会去验证这个想法。下一个剧...,卢泽决定跟随那女子，夜色渐深，医务室的门缝里透出微弱的光。他躲在角落里，心跳加速。女子四下望...,“这倒是，”卢泽赞同地点点头，“咱们今后一定要尽量避免和她在一块儿用餐，到时候她如果还不主动...,想到这里，他突然一个闪身，躲进自己的房间，并打开房门，假装无意中从那里经过，看到那女子端着一...,“不过她吃饭的时候还是必须在场。”\n “那就好办了。”玛瑟说，“到时候给她吃的东西里...
7,科幻小说,《血腥科技有限公司》,“你！？船长！！你怎么没有死！？”brent惊骇的叫道。“嘿嘿，当然，蜂蜜把我的毒给解了。”...,"""词汇选择 - 运用惊讶、轻松和戏谑交错的词汇，描绘了一个紧张却又不失幽默感的对峙场景。\n...",300,司马朗慢慢地从怀中掏出一块已经破损的照片，递给了对方，“真正的Brent在三年前就已经葬身海...,对方接过照片，手指轻轻摩挲着上面已经有些模糊的人影，仿佛想从中寻找些什么。“这…这怎么可能？...,“你是怎么知道的！？”他惊叫道。“你再仔细看看，这张照片是不是很模糊！？”司马朗问道。“是啊...,“你，你是谁！？”他结结巴巴的道。“呵呵，不愧是老奸巨滑的brent！”司马朗冷笑道，同时从...,“这张照片是我三年前在潜艇上拍下来的，很清晰，能看出你眉宇间的不同。”司马朗淡淡道。“你！！...
8,历史穿越,《逆天神针，鬼医毒王妃》,紫苏本想劝慰几句，上官晴曦却直接抬了手，“好了，什么也不用说，你们且都下去吧，我需要静静。”...,"""词汇选择 - 运用了充满情感色彩的词汇，塑造了一种深邃复杂的情感世界。\n句法结构 - 使...",400,在接下来的发展中，上官晴曦可能会回忆起过去和灵鸢的一些片断，并尝试通过这些线索来探寻灵鸢触碰...,上官晴曦的思绪似乎回到了那日的宴会，灵鸢以一袭白衣，气质冷峻，站在她的面前。她记得，灵鸢当时...,难道，是灵鸢在那时救了她？否则，她怎么会有这么清晰且深刻的感受？\n她再清楚不过自己的身体，...,她忍不住抬头看了一眼房顶，仿佛透过层层叠叠的房顶，她能够看到那张令她避之不及的脸――那个白衣...,后来，她细细回想过她与灵鸢的点点滴滴，终于想起，在她被针扎般疼痛的梦中，那个白衣女子曾说，她...
9,历史穿越,《萌女穿越记》,淼淼从云姨口中得知：这里并不是她所在的那个时空，而是一个异世，这里的发展水平相当于古代的清朝...,"""词汇选择 - 通过用词描绘异世的精细构建，营造出一种跨越时空、充满奇幻色彩的叙述氛围。\n...",200,东陵国的王子雅轩深爱着南盛国的公主婉约。然而两个国家之间对于边境的小规模冲突使两国的关系变得...,在一场宴会上，淼淼无意间听到两位不明身份的宾客窃窃私语：“听说东陵国的雅轩王子和南盛国的婉约...,而淼淼正是东陵国国主的妹妹，雅轩王爷的小妹！(22),然而一场刺杀使两国关系破裂，南盛国的公主婉约身受重伤，雅轩苦追婉约，婉约却死在雅轩的怀里，雅...,然而两国国王的死，两国新王的继承，让两个国家的关系变得更为紧张，再加上恐怖组织的从中挑拨，两...


In [6]:
# 将DataFrame写入CSV文件  
data.to_csv('res_v4.csv', index=False)  # index=False表示不在CSV文件中包含行索引